In [ ]:
import io
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
def click_yesterday_button(driver):
    button_xpath = "/html/body/div[1]/div[4]/div[2]/div/div[3]/form/table/tbody/tr[4]/td/input[2]"
    button = driver.find_element("xpath", button_xpath)
    button.click()

In [ ]:
def retrieve_date(driver):
    input_element_xpath = "/html/body/div[1]/div[4]/div[2]/div/div[3]/form/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]/input"
    input_element = driver.find_element("xpath", input_element_xpath)
    date = input_element.get_attribute("value")
    return date

In [ ]:
def retrieve_table_content(driver):
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, "html.parser")
    target_table = soup.select_one("div > div > div > div > table > tbody > tr:nth-of-type(2) > td > table:nth-of-type(1)")
    return target_table

In [ ]:
def process_table_to_rawdata(target_table, date):
    df = pd.read_html(io.StringIO(str(target_table)))[0]
    df.columns = df.iloc[2]
    df = df.drop([0, 1, 2]).reset_index(drop=True)
    df = df.iloc[:, [1, 2, -2]]
    df = df.iloc[:3]
    df_pivoted = df.pivot(index='商品 名稱', columns='身份別', values='口數').reset_index()
    df_pivoted['Date'] = date
    df_pivoted = df_pivoted[['Date'] + [col for col in df_pivoted if col != 'Date']]
    return df_pivoted

In [ ]:
def wait_for_page_load(driver, timeout=10):
    element_present = EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[4]/div[2]/div/div[3]/form/table/tbody/tr[4]/td/input[2]"))
    WebDriverWait(driver, timeout).until(element_present)

In [ ]:
service = Service(executable_path='C:/chromedriver/chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
URL = "https://www.taifex.com.tw/cht/3/futContractsDate"
driver.get(URL)

In [ ]:
for i in range(1000):
    date = retrieve_date(driver)
    target_table = retrieve_table_content(driver)

    if target_table:
        df_pivoted = process_table_to_rawdata(target_table, date)

        with open('TaiwanFuturesExchange.csv', 'a') as f:
            df_pivoted.to_csv(f, header=False, index=False)

    click_yesterday_button(driver)
    wait_for_page_load(driver)

In [ ]:
driver.quit()

In [ ]:
import io
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, NoSuchWindowException, WebDriverException

def click_yesterday_button(driver):
    button_xpath = "/html/body/div[1]/div[4]/div[2]/div/div[3]/form/table/tbody/tr[4]/td/input[2]"
    try:
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, button_xpath))
        )
        button.click()
    except (TimeoutException, NoSuchWindowException, WebDriverException) as e:
        print(f"Exception occurred while clicking yesterday button: {e}")
        driver.refresh()  # 刷新页面，重新加载
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, button_xpath))
        ).click()

def retrieve_date(driver):
    input_element_xpath = "/html/body/div[1]/div[4]/div[2]/div/div[3]/form/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]/input"
    try:
        input_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, input_element_xpath))
        )
        date = input_element.get_attribute("value")
        return date
    except (TimeoutException, NoSuchWindowException, WebDriverException) as e:
        print(f"Exception occurred while retrieving date: {e}")
        return None

def retrieve_table_content(driver):
    try:
        page_content = driver.page_source
        soup = BeautifulSoup(page_content, "html.parser")
        target_table = soup.select_one("div > div > div > div > table > tbody > tr:nth-of-type(2) > td > table:nth-of-type(1)")
        return target_table
    except Exception as e:
        print(f"Error retrieving table content: {e}")
        return None

def process_table_to_rawdata(target_table, date):
    try:
        df = pd.read_html(io.StringIO(str(target_table)))[0]
        df.columns = df.iloc[2]
        df = df.drop([0, 1, 2]).reset_index(drop=True)
        df = df.iloc[:, [1, 2, -2]]
        df = df.iloc[:3]
        df_pivoted = df.pivot(index='商品 名稱', columns='身份別', values='口數').reset_index()
        df_pivoted['Date'] = date
        df_pivoted = df_pivoted[['Date'] + [col for col in df_pivoted if col != 'Date']]
        return df_pivoted
    except Exception as e:
        print(f"Error processing table to raw data: {e}")
        return None

def wait_for_page_load(driver, timeout=10):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[4]/div[2]/div/div[3]/form/table/tbody/tr[4]/td/input[2]"))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Timeout waiting for page to load")

service = Service(executable_path='C:/chromedriver/chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

URL = "https://www.taifex.com.tw/cht/3/futContractsDate"
driver.get(URL)

for i in range(1000):
    date = retrieve_date(driver)
    if date:
        target_table = retrieve_table_content(driver)
        if target_table:
            df_pivoted = process_table_to_rawdata(target_table, date)
            if df_pivoted is not None:
                with open('TaiwanFuturesExchange.csv', 'a') as f:
                    df_pivoted.to_csv(f, header=False, index=False)
    click_yesterday_button(driver)
    wait_for_page_load(driver)

driver.quit()
